# Test

In [42]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('..')

from sportsbooks import (
    pinnacle,
    draftkings,
    pointsbet
)

from calculator import Calculator

In [43]:
league=487

pinny = pinnacle.Pinnacle(league=league)
pinny.get_data()

In [44]:
dkng = draftkings.DraftKings(
    league=league, 
    category_id=4606
)

dkng.get_data()

In [45]:
pb = pointsbet.PointsBet()

pb.get_data()

In [46]:
retail = (
    dkng.df
    .assign(
        DraftKings = lambda x: x['price'].apply(Calculator.convert_american_to_decimal)
    )
    [['participant_name', 'points', 'DraftKings']]
    .merge(
        (
            pb.df
            .rename(columns={'odds_decimal': 'PointsBet'})
        )
    )
    .melt(
        id_vars=['participant_name', 'points'],
        var_name='book',
        value_name='Decimal Odds'
    )
)

In [48]:
ROI = (
    pinny.df
    .assign(
        raw_probability = lambda x: x['price'].apply(Calculator.get_implied_probability),
        vig_free_probability = lambda x: x['raw_probability'] / x.groupby(['matchup_id', 'key'])['raw_probability'].transform('sum')
    )
    [['participant_name', 'points', 'price', 'vig_free_probability']]
    .merge(
        retail,
        on=['participant_name', 'points'],
        how='left'
    )
    .assign(
        ROI = lambda x: round(100 * (x['Decimal Odds'] * x['vig_free_probability'] - 1),1),
        AmericanOdds = lambda x: round(x['Decimal Odds'].apply(Calculator.convert_decimal_to_american))
    )
    .sort_values(
        by='ROI',
        ascending=False
    )
    .head(15)
)

ROI

,participant_name,points,price,vig_free_probability,book,Decimal Odds,ROI,AmericanOdds
188,Dallas Mavericks,4.5,-165,0.603056,PointsBet,1.666700,0.5,-150.0
191,Dallas Mavericks,3.5,-142,0.569012,PointsBet,1.740800,-0.9,-135.0
193,Dallas Mavericks,2.5,-120,0.530316,DraftKings,1.869565,-0.9,-115.0
79,Oklahoma City Thunder,-7.5,113,0.455417,PointsBet,2.150000,-2.1,115.0
47,Los Angeles Lakers,2.5,138,0.406914,DraftKings,2.400000,-2.3,140.0
190,Dallas Mavericks,3.5,-142,0.569012,DraftKings,1.714286,-2.5,-140.0
70,Oklahoma City Thunder,-4.5,-146,0.575041,DraftKings,1.689655,-2.8,-145.0
71,Oklahoma City Thunder,-4.5,-146,0.575041,PointsBet,1.689700,-2.8,-145.0
246,San Antonio Spurs,4.5,-146,0.575041,PointsBet,1.689700,-2.8,-145.0
194,Dallas Mavericks,2.5,-120,0.530316,PointsBet,1.833400,-2.8,-120.0


In [49]:
ROI

,participant_name,points,price,vig_free_probability,book,Decimal Odds,ROI,AmericanOdds
188,Dallas Mavericks,4.5,-165,0.603056,PointsBet,1.666700,0.5,-150.0
191,Dallas Mavericks,3.5,-142,0.569012,PointsBet,1.740800,-0.9,-135.0
193,Dallas Mavericks,2.5,-120,0.530316,DraftKings,1.869565,-0.9,-115.0
79,Oklahoma City Thunder,-7.5,113,0.455417,PointsBet,2.150000,-2.1,115.0
47,Los Angeles Lakers,2.5,138,0.406914,DraftKings,2.400000,-2.3,140.0
190,Dallas Mavericks,3.5,-142,0.569012,DraftKings,1.714286,-2.5,-140.0
70,Oklahoma City Thunder,-4.5,-146,0.575041,DraftKings,1.689655,-2.8,-145.0
71,Oklahoma City Thunder,-4.5,-146,0.575041,PointsBet,1.689700,-2.8,-145.0
246,San Antonio Spurs,4.5,-146,0.575041,PointsBet,1.689700,-2.8,-145.0
194,Dallas Mavericks,2.5,-120,0.530316,PointsBet,1.833400,-2.8,-120.0


In [55]:
print(ROI.to_markdown(index=False))

| participant_name       |   points |   price |   vig_free_probability | book       |   Decimal Odds |   ROI |   AmericanOdds |
|:-----------------------|---------:|--------:|-----------------------:|:-----------|---------------:|------:|---------------:|
| Dallas Mavericks       |      4.5 |    -165 |               0.603056 | PointsBet  |        1.6667  |   0.5 |           -150 |
| Dallas Mavericks       |      3.5 |    -142 |               0.569012 | PointsBet  |        1.7408  |  -0.9 |           -135 |
| Dallas Mavericks       |      2.5 |    -120 |               0.530316 | DraftKings |        1.86957 |  -0.9 |           -115 |
| Oklahoma City Thunder  |     -7.5 |     113 |               0.455417 | PointsBet  |        2.15    |  -2.1 |            115 |
| Los Angeles Lakers     |      2.5 |     138 |               0.406914 | DraftKings |        2.4     |  -2.3 |            140 |
| Dallas Mavericks       |      3.5 |    -142 |               0.569012 | DraftKings |        1.71429 |  